# QCArchive Workshop

This notebook shows you how to get started with QCArchive - a MolSSI software product that helps you run quantum chemistry calculations and archive the results.  In most cases, QCArchive would be set up on a server for your reserach group or team, and access to the calculations and the results would be limited to whoever was granted access to your server.  In this tutorial, we will be using the MolSSI QCArchive demonstration server, so that you can get a feel for what QCArchive can do.  

QCArchive actually has multiple parts, including the QCPortal, which is how you interact with the server.  We will be using QCPortal and some of its features in this tutorial.

## Installing QCPortal
You have to intall QCPortal on your computer to use it to interact with the QCArchive Server.   We maintain a QCPortal environment file that you can install with the acadonda API, that will create an enviroment and install all the required components for QCArchive.  To use this, you need to have the anaconda-client installed in your base conda environment.  

First install anaconda-client in your base environment, using this command.  (If it is already installed, it will just tell you it is already installed.) <br>
`conda install anaconda-client -n base`

Then create the qcarchive_demo enviorment and install all required software with this command <br>
`conda env create qcarchive/qcarchive_demo`

Now activate your conda enviroment, and then restart this notebook. <br>
`conda activate qcarchive_demo`

In [ ]:
import qcportal as ptl
from qcportal.molecules import Molecule

import nglview

## Create a client object and connect to the demo server

The `PortalClient` is how you interact with the server, including querying records and submitting computations.

The demo server allows for unauthenticated guest access, so no username/password is necessary to read from the server. However, you will
need to log in to submit or modify computations.  If you are taking a QCArchive workshop, you will be provided a username and password.  If not, please contact the MolSSI (qcarchive@molssi.org) to get a demo username and password if you would like to try out the software.  The demo server is intended for testing and demonstration use only, not any significant calculations.

In [ ]:
# This is the way you would connect without authentication if you were only going to read information.
#client = ptl.PortalClient("https://qcademo.molssi.org")

# This is how you long in with a username and password.
client = ptl.PortalClient("https://qcademo.molssi.org", username="qcademo", password="molssi_qcademo")

## Single-point energy calculations for one molecule

### Specifying the molecule

We are going to begin by creating one molecule and doing a singlepoint energy calculation.  

The part of QCArchive that creates and manages molecules is called QCElemental.  There are several ways you can create a molecule.  If the molecule is simple enough, you can create it by directly specifiying the geometry with xyz coordinates.  This is hard unlesse it is a very simple molecule.

This is a hydrogen molecule with a bond distance of 1.0 bohr.

In [ ]:
# Most direct way to make a molecule
hydrogen = Molecule(symbols=['h', 'h'], geometry=[[0.0, 0.0, 0.0, 0.0, 0.0, 1.0]])

Another way is to specify an xyz file as a string and the generate the molecule from that string.  

The triple quotes notation in python lets you specify multi-line strings.

In [ ]:
#Another way to specify molecule is to create a string of an xyx file
mol_xyz = """
  3

  H   0.000000 1.000000 0.000000
  H   1.000000 0.000000 0.000000
  O   0.000000 0.000000 0.000000
"""
water = Molecule.from_data(mol_xyz)
print(water)

In [ ]:
# Another way is to have read in an xyz file that you already have.  
water2 = Molecule.from_file('water.xyz')
print(water2)

### Choosing job options and submitting the computation

To create our computations, there are different methods to add different types of computations.  For example, for an energy caluation, the method is `add_singlepoints`.  The methods for adding computations (like `add_singlepoints`) take a list of molecules and the details of how to run the computation. In this example, we only have one molecule for now.

When we execute a method command, this sends it to the server, where it is queued for computation. This will eventually be picked up by a worker if there is a worker that can handle it (proper programs installed, etc).

Here are the options you will specify for your computation:
* **program** = use this program to run the computation
* **driver** = the main type of computation to run (energy, gradient, properties, etc)
* **method**/**basis** = the computational model to use

All `add_` functions return two objects. The first is metadata about the addition, and second is a list of record ids - we will only have one because we only submitted one molecule.
We can use these IDs later to retrieve the record, which will let us look at our results.

QCArchive supports a lot of different QM calculations programs.  The input to QCPortal is exacxtly the same no matter what QM program you are using. 

In [ ]:
qmprogram = 'psi4'
calc_type = 'energy'
calc_method = 'b3lyp'
basis_set = 'def2-tzvp'
# Note the first input to this function is the molecules that you want to calculate.  
# Note that this must be a list, so we have
# to use the square brackets, even if there is only one thing in the list.
molecule_list = [water2]
meta, record_ids = client.add_singlepoints(molecule_list, 
                                           program=qmprogram, 
                                           driver=calc_type, 
                                           method=calc_method, 
                                           basis=basis_set)

#Print the list of record IDs that we will use to access the data later.
# This will be a list of the same length as your molecule list
print(record_ids)

## Looking at the results
Now we need to look at the results from our calculation.  To do this, we need to create a `record` of our calcluation results so that then we can pull out different information.  To create a record use `client.get_records` and input the record_id.  Remember that record_id is a list, so you have to use the list syntax.

### Information from the calculation record

In [ ]:
water_record = client.get_records(record_ids[0])

This doesn't print anything, but now we have a `record` object that we can use to access different information about our calculation.  We can access a variety of information and the types of information we can access from the record depends on what kind of calculation we did. We will demonstate several kinds of information you can access from `record` in the new few cells.

In [ ]:
# You can print the status of the calculation to see if it is finished running.
print(water_record.status)

In [ ]:
#Show the entire output file.  This is probably not what we are interested in, but you can do it.
# print(water_record.stdout)

In [ ]:
# You can print the information about your molecule
print(water_record.molecule)

You can use this molecule information to visualize in nglview.

In [ ]:
display(water2)

In [ ]:
# You can print the details of your calculation.
print(water_record.specification)

Another thing you can access from the `record` is the provenance of your calculation.  This contains even more information than the specification, including the exact version of the software used to run the calculation and the walltime.  The syntax is slightly more complicated because you have to accesse the last element of the compute history and then the provenance from that.

In [ ]:
# Show the information available in the provenance
print(water_record.compute_history[-1].provenance)

In [ ]:
# Access a particular peice of information from the provenance
time_s = water_record.compute_history[-1].provenance.wall_time
print(time_s)

### Information from the calculation: molecular properties

The things we have looked at from `record` so far are mostly details about the calculation itself and not really the results of the calculation.  Most of the information about the results (i.e. the properties of your molecule that you calculated) is stored in a python dictionary that you can access from `record`.  Remember that a python dictionary is a data structure where you look up information by a key (a word) instead of an index (a number).  First we extract the dictionary from the records and give it a name.  Then we can see all the keys, which are all the molecular properties we could access.  Then we can print or do any other operation with any of those properties.

In [ ]:
# Extracting the dictionary from the record and giving it a name
water_properties = water_record.properties
# Looking at all the keys for the dictionary to see what information is stored
print(water_properties.keys())

In [ ]:
#Pick a property you want to look at and print it
energy = water_properties['scf_total_energy']
print(energy)
# You could do this in one line if you wanted to like this
print(water_properties['scf_total_energy'])

### Multiple calculations for one molecule
We can iterate over multiple calculation types for one molecule (or many molecules, 
but we will get to that). Let's add some additional computational methods for our water calculation.


In [ ]:
qmprogram = 'psi4'
calc_type = 'energy'
#Change calc method to a list
calc_method = ['b3lyp', 'm06-2x', 'pbe0']
basis_set = 'def2-tzvp'
# Don't forget list notation!
molecule_list = [water2]

#Set up empty lists to hold the meta data and record_ids
record_ids = []

for functional in calc_method:
    meta, record_id = client.add_singlepoints(molecule_list, 
                                           program=qmprogram, 
                                           driver=calc_type, 
                                           method=functional, 
                                           basis=basis_set)
    print(meta, record_id)
    record_ids.extend(record_id)

### Exercise
1. Find the record IDs for your three calculations.
2. Check to see if they are finished.
3. Print the energy for each method in a neatly formatted print statement.

### Extension Exercise
Do a basis set convergence study.  Increase the basis set size for one molecule.  Look at the energies to see if the energy has converged with respect to basis set.  Then look at the walltime to see how much more expensive the bigger calculations were.

In [ ]:
# Write your code here



    
    
    
    

## More complex calculations with QCSpecification

If you are doing any calculation more complicated than a single-point calculation, then we need to use a QCSpecification.  The QCSpecification is the details of the QM calculation that you want to run.  The overall QCArchive ecosystem is structured in a hierarchical way, where a single point calculation is the simplest type.  You can think of a geometry optimization as numerous single-point calculations, so the specification is the details of one of those single-point calculations. (Later, when we want to create a dataset by running the same kind of calculation for many molecules, we will also need to use a QCSpecification.)  In this example, we will do a geometry optimization calculation, which will first require setting a specification.

In [ ]:
# Import the QCSpecification module
from qcportal.singlepoint import QCSpecification

In [ ]:
# Set up a water calculation with a stretched bond
water_xyz = """
  3

  H   0.000000 1.500000 0.000000
  H   1.000000 0.000000 0.000000
  O   0.000000 0.000000 0.000000
"""

water_stretched = Molecule.from_data(water_xyz)

In [ ]:
#Visualize the molecule
display(water_stretched)

Now we will set up our calculation details using QCSpecification.  Then when we use `add_optimization` to set up our calculation, we will give our specification instead of having to enter in all the computation details.

In [ ]:
# Set the specification
spec = QCSpecification(program="psi4", 
                       driver="deferred", 
                       method='hf', 
                       basis='sto-3g', 
                       keywords={'maxiter': 100})

# Submit the geometry optimization
meta, id = client.add_optimizations(initial_molecules=[water_stretched], 
                                    program="geometric",
                                    qc_specification=spec)

In [ ]:
# Get the record for this calculation
opt_record = client.get_records(id[0])

In [ ]:
# View the optimized geometry
optimized_geometry = opt_record.final_molecule
display(optimized_geometry)

In [ ]:
# Save the optimized geometry to a file
optimized_geometry.to_file('water_opt.xyz')

## Data Sets
A data set is a set of one kind of calculation (single points, optimizations, etc.) for many different molecules (called entries in a QCArchive dataset).  Think of it like a table where the rows are the molecules and the columns are the different QM calculation methods (the specifications) you want to run. A cell within the table (intersection between a molecule and a specificiation) is a record that has the output for the molecule for that particular specification. 

|       | HF/sto-3g | B3LYP/def2-tzvp | MP2/cc-pvdz |
| ---   | ---       | ---             | ---         |
| water | 18263     | 18277           | 18295       |
|methane| 19722     | 19642           | 19867       |
|ethanol| 20212     | 20931           | 23210       |



### Creating a dataset
In this example, we will create a singepoint energy data set for several elements.  Remember, a dataset can only contain one type of calculation.

Note: Within your archive, you can not have more than one dataset with the same name.  Give your dataset in the example below a unique name.  Note that once you have run the cell, your dataset will exist, so if you try to run this cell again, it will tell you the dataset already exists.


In [ ]:
ds = client.add_dataset("singlepoint",
                        name="   ",
                        description="Variety of calculations on single atoms")

Now we will make a list of our favorite elements, and create a molecule object for each one.  We will specify that the coordinates for each atom is at the origin.  For each molecule, we will add it as an entry in our dataset using `ds.add_entry`.  We must give each entry a name, which is a string.  We will give each entry the name `X_atom` where X is the element's symbol.

In [ ]:
# Choose a few of your favorite elements
favorite_elements = [H, C, Si, Ag]
for element in favorite_elements:
    mol = Molecule(symbols=[element], geometry=[0.0, 0.0, 0.0])
    
    # Creates an entry from the molecule. The entry contains the molecule and a name,
    # but there are additional fields you can have as well
    entry_name = element + "_atom"
    ds.add_entry(name=entry_name, molecule=mol)

We will now create two different specifications, and add them to the dataset. The first will be hf/sto-3g, and the second will be b3lyp/aug-cc-pvtz.

In this example, we will use the same QM program, but you could use different specifications to compare programs, different functionals, basis sets, etc.

For both calculations, we will increase the maximum number of SCF iterations to 100

In [ ]:
spec_1 = QCSpecification(
            program="psi4",
            driver="energy",
            method="hf",
            basis="sto-3g",
            keywords={"maxiter": 100}
)

spec_2 = QCSpecification(
            program="psi4",
            driver="properties",
            method="b3lyp",
            basis="aug-cc-pvtz",
            keywords={"maxiter": 100}
)

ds.add_specification(name="hf/sto-3g", specification=spec_1)
ds.add_specification(name="b3lyp/aug-cc-pvtz", specification=spec_2)

### Submitting the computations and checking the status
At this point, we have added specifications and entries,
but have not submitted any calculations yet. We do that with
the `submit()` function

By default, this submits all calculations, but you could choose to only submit certain entries (particular molecules) or certain specifications (certain calculation types).

In [ ]:
ds.submit()

We can check the status of our dataset.

In [ ]:
# The basic way to check the status is
ds.status()
# A better formatted print is
ds.print_status()

### Accessing results for calculations in a dataset
If you want to access the results of one of these calculations, you need to create a record for the one you want to view.  Previously, we did this with the record id number, but now I don't know that record id for a particular calculation.  Instead, in a dataset, you can access the record by giving the entry name and the specification name, still using the `get_record` option.

In [ ]:
# Create a record for a particular entry/specification pair
rec = ds.get_record('Ag_atom', 'hf/sto-3g')
# I can print the record id for this entry in the data set
print(rec.id)
# I can print 
print(rec.properties['return_energy'])

### Exercise
Create a new specification and add it to your data set.  You can also add new entries (additional atoms) if you wish.  Rerun the dataset and check the status.  

In [ ]:
# Write your code here



### Extension Exercise
For one of your entries (one of the atoms), compare the results of your new specification to the results for the b3lyp/aug-cc-pvtz calculation.

In [ ]:
# Write your code here


### Accessing results for all the calculations in a dataset

To get results for all the calclulations in a dataset, you can use the function `iterate_records` within a `for` loop.  This function iterates over three things: the entry, the specification, and the record.  You can give optional inputs to the `interate_records` function to only iterate over the calculations that are complete or only look at a particular specification, or other options.  

In this example, we will print the entry name, the specification, and the final energy

In [ ]:
#e - entry 
#s - specification 
#r - record
# We are only going to print the results for calculations that are complete.
for e, s, r in ds.iterate_records(status='complete'):
    # Get the energy from the record
    # Remember it is in the properties dictionary that is accessed from the record
    energy = r.properties['return_energy']
    # Print this for everything in the dataset
    print(e, s, energy)

You could use formatted print statements to make this look better, or you could store these values in lists or arrays or any other type of python data strcture.

If you look carefully, you notice that there are no results printed for the entry `Ag_atom` for the specification `b3lyp/aug-cc-pvtz`.  This is probably the calculation that had an error, therefore its status is not complete.  We will come back to this later.

### Compiling results into a dataframe

A more advanced way to analyze your results is to compile them into a dataframe, such that you could now use any of the querying and sorting capabilities of pandas.  To do this, we will use a function called `compile_values`.  The inputs of `compile_values` are a python callable (generally a function) and a string that gives a name for the data.  The output is a pandas dataframe. 

In general, you write a function that takes a record as an input and returns whatever data you want that you can access from the record.  Then you input that function into `compile_values` as the callable.

In [ ]:
#Write a function to extract the data you want from the record
def get_energy(record):
    # Get the energy from the properties dictionary
    energy = record.properties['return_energy']
    return energy

# Syntax: dataset_name.compile_values(callable, name_string)
# The output is a dataframe
energies_df = ds.compile_values(get_energy, 'total_energy')
    
# If you know what a lambda is you can do it this way    
#df = ds.compile_values(lambda r: r.properties["return_energy"], 'total_energy')

In [ ]:
# Show the dataframe
energies_df

We again notice that the results for Ag_atom for b3lyp/aug-cc-pvtz is NaN.  This must be the calculation that had an error when we checked the status above.  A calculation that ends in an error still has a record.  Let's investigate the record for the failed calculation.

In [ ]:
# Let's see why this is NaN
# Get the record for the calculation that gave NaN results
Ag_b3lyp_record = ds.get_record("Ag_atom", "b3lyp/aug-cc-pvtz")
# Print the status
print(Ag_b3lyp_record.status)

Every record has a dictionary called `error`.  We can use the key `error_message` to access the error message from the dictionary.  It prints a lot of information, but generally, you can find the problem near the end of the message.

In [ ]:
# From the error, access the error dictionary and print the error message
print(Ag_b3lyp_record.error['error_message'])

Perhaps unsurprinstly, there is no basis function for silver in the aug-cc-pvtz basis set.